# Pandas -- Series and DataFrames


Main source: https://github.com/jakevdp/PythonDataScienceHandbook/tree/master/notebooks

#### Pandas is a library for fast and efficient computation on big datasets. As in Numpy, many operations in Pandas are vectorized and thus efficient and fast.
Pandas is a newer package built on top of NumPy, and provides an efficient implementation of a DataFrame. DataFrames are essentially multidimensional arrays with attached row and column labels, and often with heterogeneous types and/or missing data. As well as offering a convenient storage interface for labeled data, Pandas implements a number of powerful data operations familiar to users of both database frameworks (-> relational algebra) and spreadsheet programs.

As we saw, NumPy's ndarray data structure provides essential features for the type of clean, well-organized data typically seen in numerical computing tasks. While it serves this purpose very well, its limitations become clear when we need more flexibility (e.g., attaching labels to data, working with missing data, etc.) and when attempting operations that do not map well to element-wise broadcasting (e.g., groupings, pivots, etc.), each of which is an important piece of analyzing the less structured data available in many forms in the world around us. Pandas, and in particular its Series and DataFrame objects, builds on the NumPy array structure and provides efficient access to these sorts of "data munging" tasks that occupy much of a data scientist's time.

In [1]:
# Just as we import numpy usually as np, we import pandas under the alias of pd. 
# We'll import numpy as well, because we'll need it often when using pandas
import numpy as np
import pandas as pd

## The Pandas Series Object
A Pandas Series is a one-dimensional array of indexed data. It can be created from a list or array as follows:

In [2]:
data = pd.Series([0.25, 5, np.NaN, 1.0])
data

0    0.25
1    5.00
2     NaN
3    1.00
dtype: float64

In [3]:
type(data)

pandas.core.series.Series

In [4]:
data.values, type(data.values)

(array([0.25, 5.  ,  nan, 1.  ]), numpy.ndarray)

In addition to the values Pandas Series have an index which is an array-like object

In [5]:
data.index, type(data.index), list(data.index)

(RangeIndex(start=0, stop=4, step=1),
 pandas.core.indexes.range.RangeIndex,
 [0, 1, 2, 3])

Like with a NumPy array, data can be accessed by the associated index via the familiar Python square-bracket notation:

In [6]:
data[1:3]

1    5.0
2    NaN
dtype: float64

In [7]:
type(data[1])

numpy.float64

### Series as generalized NumPy array

From what we've seen so far, it may look like the Series object is basically interchangeable with a one-dimensional NumPy array. The essential difference is the presence of the index: while the Numpy Array has an implicitly defined integer index used to access the values, the Pandas Series has an explicitly defined index associated with the values.

This explicit index definition gives the Series object additional capabilities. For example, the index need not be an integer, but can consist of values of any desired type. For example, if we wish, we can use strings as an index:

In [8]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'd', 'c'])
data

a    0.25
b    0.50
d    0.75
c    1.00
dtype: float64

In [9]:
data.index = list("AbCD")
data

A    0.25
b    0.50
C    0.75
D    1.00
dtype: float64

In [10]:
data["b"] == data[1] == data.values[1]

True

In [11]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=[3, 7, 2, 4])
data

3    0.25
7    0.50
2    0.75
4    1.00
dtype: float64

In [12]:
data.index

Int64Index([3, 7, 2, 4], dtype='int64')

When an explicit index is present, it is preferred! (*as long as we don't slice!*)

In [13]:
data[3]

0.25

In [14]:
data[1:3]

7    0.50
2    0.75
dtype: float64

Note that explicit indices don't need to be unique!

In [15]:
d2 = pd.concat([data, data])
d2

3    0.25
7    0.50
2    0.75
4    1.00
3    0.25
7    0.50
2    0.75
4    1.00
dtype: float64

In [16]:
d2[3]

3    0.25
3    0.25
dtype: float64

In [17]:
d2[3] = 2
d2

3    2.00
7    0.50
2    0.75
4    1.00
3    2.00
7    0.50
2    0.75
4    1.00
dtype: float64

### Series as specialized dictionary

In this way, you can think of a Pandas Series a bit like a specialization of a Python dictionary. A dictionary is a structure that maps arbitrary keys to a set of arbitrary values, and a Series is a structure which maps typed keys to a set of typed values. This typing is important: just as the type-specific compiled code behind a NumPy array makes it more efficient than a Python list for certain operations, the type information of a Pandas Series makes it much more efficient than Python dictionaries for certain operations.

The Series-as-dictionary analogy can be made even more clear by constructing a Series object directly from a Python dictionary:

In [18]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)
population

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

In [19]:
population['Texas']

26448193

Unlike a dictionary, though, the Series also supports array-style operations such as slicing:

In [20]:
population['California':'Illinois']
# note that Illinois is included!

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

### Constructing Series objects

In [21]:
# data can be a list or numpy array
pd.Series([2,4,5,6,7,8], index=list("abcdef"))

a    2
b    4
c    5
d    6
e    7
f    8
dtype: int64

In [22]:
# data can be a scalar
pd.Series(5, index=[100, 200, 300])

100    5
200    5
300    5
dtype: int64

In [23]:
# data can be a dictionary
ser = pd.Series({2:'a', 1:'b', 3:'c'})
ser

2    a
1    b
3    c
dtype: object

Notice that in this case, the Series is populated only with the explicitly identified keys:

In [24]:
pd.Series({2:'a', 1:'b', 3:'c'}, index=[3, 2])

3    c
2    a
dtype: object

<br>

## The Pandas DataFrame Object

The next fundamental structure in Pandas is the DataFrame. Like the Series object, it can be thought of either as a generalization of a NumPy array, or as a specialization of a Python dictionary. We'll now take a look at each of these perspectives.

### DataFrame as a generalized NumPy array

If a Series is an analog of a one-dimensional array with flexible indices, a DataFrame is an analog of a two-dimensional array with both flexible row indices and flexible column names. Just as you might think of a two-dimensional array as an ordered sequence of aligned one-dimensional columns, you can think of a DataFrame as a sequence of aligned Series objects. Here, by "aligned" we mean that they share the same index.



To demonstrate this, let's first construct a new Series listing the area of each of the five states discussed in the previous section:

In [25]:
area_dict = {'California': 423967, 
             'Texas': 695662, 
             'New York': 141297,
             'Florida': 170312, 
             'Illinois': 149995}

area = pd.Series(area_dict)
area

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
dtype: int64

Now that we have this along with the population Series from before, we can use a dictionary to construct a single two-dimensional object containing this information:

In [26]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}

population = pd.Series(population_dict)
population

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

In [27]:
states = pd.DataFrame({'population': population,
                       'area': area,
                       'country': 'USA'})
states

,population,area,country
California,38332521,423967,USA
Texas,26448193,695662,USA
New York,19651127,141297,USA
Florida,19552860,170312,USA
Illinois,12882135,149995,USA


In [28]:
print(states.dtypes)

population     int64
area           int64
country       object
dtype: object


This looks like a generalized dictionary! The keys are the names of the state, and the values are like a list [area, country, population]

In [29]:
states.sort_values(by="population", ascending=False)

,population,area,country
California,38332521,423967,USA
Texas,26448193,695662,USA
New York,19651127,141297,USA
Florida,19552860,170312,USA
Illinois,12882135,149995,USA


In [30]:
states['population'], type(states['population'])

(California    38332521
 Texas         26448193
 New York      19651127
 Florida       19552860
 Illinois      12882135
 Name: population, dtype: int64,
 pandas.core.series.Series)

In [31]:
states["population"].idxmax() #figures out the "key(s)"(indices) of the DataFrame where "population" has its max

'California'

In [32]:
states.loc[states["population"].idxmax()] #returnes the series at the given index

population    38332521
area            423967
country            USA
Name: California, dtype: object

In [33]:
states.max() #returns the max for every individual column!

population    38332521
area            695662
country            USA
dtype: object

In [34]:
states['California']

In [35]:
states.loc['California']

population    38332521
area            423967
country            USA
Name: California, dtype: object

In [36]:
states.index

Index(['California', 'Texas', 'New York', 'Florida', 'Illinois'], dtype='object')

In [37]:
states.columns

Index(['population', 'area', 'country'], dtype='object')

In [38]:
states.values

array([[38332521, 423967, 'USA'],
       [26448193, 695662, 'USA'],
       [19651127, 141297, 'USA'],
       [19552860, 170312, 'USA'],
       [12882135, 149995, 'USA']], dtype=object)

In [39]:
type(states.values)

numpy.ndarray

Thus the DataFrame can be thought of as a generalization of a two-dimensional NumPy array, where both the rows and columns have a generalized index for accessing the data.

## DataFrame as specialized dictionary

Similarly, we can also think of a DataFrame as a specialization of a dictionary. Where a dictionary maps a key to a value, a DataFrame maps a column name to a Series of column data. For example, asking for the 'area' attribute returns the Series object containing the areas we saw earlier:

In [40]:
states["area"]
# note that indexing a DataFrame with square brackets gets the *column*!

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [41]:
type(states["area"])

pandas.core.series.Series

## Constructing DataFrame objects

A Pandas DataFrame can be constructed in a variety of ways:

#### From a single Series object

A DataFrame is a collection of Series objects, and a single-column DataFrame can be constructed from a single Series:

In [42]:
population

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

In [43]:
pd.DataFrame(population, columns=['population'])

,population
California,38332521
Texas,26448193
New York,19651127
Florida,19552860
Illinois,12882135


#### From multiple Series

In [44]:
s1 = pd.Series(list("abcd"))
s2 = pd.Series(list("efgh"))
df = pd.concat([s1, s2], axis="columns")
df

,0,1
0,a,e
1,b,f
2,c,g
3,d,h


Note that many functions in pandas take the `axis` argument - In which case you can select between 0/`index` and  1/`columns`. If you want to be explicit, I recommend using the string-version!

#### From a list of dicts 

Any list of dictionaries can be made into a DataFrame. We'll use a simple list comprehension to create some data. Even if some keys in the dictionary are missing, Pandas will fill them in with NaN (i.e., "not a number") values:

In [45]:
df = pd.DataFrame([{'a': 1, 'b': 2}, {'b': 3, 'c': 4}], index=["first_dict", "second_dict"])
df

,a,b,c
first_dict,1.0,2,NaN
second_dict,NaN,3,4.0


As every single column must have a consistent dtype and np.NaN is a float, some of the numbers get coerced into floats:

In [46]:
df['a']

first_dict     1.0
second_dict    NaN
Name: a, dtype: float64

In [47]:
df['b']

first_dict     2
second_dict    3
Name: b, dtype: int64

In [48]:
df.dtypes

a    float64
b      int64
c    float64
dtype: object

If we wanted to get the rows, pandas would need to coerce the numbers explicitly: 

In [49]:
df

,a,b,c
first_dict,1.0,2,NaN
second_dict,NaN,3,4.0


In [50]:
df.loc['first_dict']

a    1.0
b    2.0
c    NaN
Name: first_dict, dtype: float64

#### From a two-dimensional NumPy array

Given a two-dimensional array of data, we can create a DataFrame with any specified column and index names. If omitted, an integer index will be used for each:

In [51]:
dates = pd.date_range('20130101', periods=6)
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))
df

,A,B,C,D
2013-01-01,0.315095,-0.885522,0.963951,-0.965382
2013-01-02,-0.528961,-1.059811,-0.238579,1.747084
2013-01-03,0.413398,-1.524741,0.560387,0.699611
2013-01-04,1.206868,0.036655,0.251522,1.215644
2013-01-05,0.017002,0.063575,-0.064048,0.524104
2013-01-06,-1.590317,-0.476948,-1.498814,0.547499


### Exercise

Create a DataFrame from the given dictionary as well as the 'qualifies' list, with the given indices:

In [52]:
exam_data = {
    'name': ['Anastasia', 'Dima', 'Katherine', 'James', 'Emily', 'Michael', 'Matthew', 'Laura', 'Kevin', 'Jonas'],
    'score': [12.5, 9, 16.5, np.nan, 9, 20, 14.5, np.nan, 8, 19],
    'attempts': [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],
    }
qualifies = ['yes', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'yes']

labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

Solutions can be found below the videos

## The Pandas Index Object

We have seen here that both the Series and DataFrame objects contain an explicit index that lets you reference and modify data. This Index object is an interesting structure in itself, and it can be thought of as an immutable array:

In [11]:
ind = pd.Index([2, 3, 5, 7, 11])
ind

Int64Index([2, 3, 5, 7, 11], dtype='int64')

In [12]:
ind[0] = 1

TypeError: Index does not support mutable operations

In [13]:
sr = pd.Series(0, index=ind)
sr

2     0
3     0
5     0
7     0
11    0
dtype: int64

Index objects have a name:

In [14]:
ind.names = ['indexx']
ind

Int64Index([2, 3, 5, 7, 11], dtype='int64', name='indexx')

In [15]:
sr = pd.Series(np.zeros_like(ind), index=ind)
sr

indexx
2     0
3     0
5     0
7     0
11    0
dtype: int64

In [16]:
df = pd.DataFrame(np.zeros_like(ind), index=ind, columns=['first'])
df

,first
indexx,
2,0
3,0
5,0
7,0
11,0


In [17]:
df.index.names = [None]
df

,first
2,0
3,0
5,0
7,0
11,0


Index objects also have many of the attributes familiar from NumPy arrays:

In [18]:
ind.size, ind.shape, ind.ndim, ind.dtype

(5, (5,), 1, dtype('int64'))

While viewing Indices as immutable list is natural, indices also allow for set-operations:

In [19]:
indA = pd.Index([1, 3, 5, 7, 9])
indB = pd.Index([2, 3, 5, 7, 11])

In [20]:
indA & indB

/var/folders/65/kg2bqjyn62n9nyh0_0s86vmc0000gn/T/ipykernel_8017/144964795.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  indA & indB


Int64Index([3, 5, 7], dtype='int64')

In [21]:
indA | indB

/var/folders/65/kg2bqjyn62n9nyh0_0s86vmc0000gn/T/ipykernel_8017/3251124211.py:1: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  indA | indB


Int64Index([1, 2, 3, 5, 7, 9, 11], dtype='int64')

In [22]:
indA ^ indB

/var/folders/65/kg2bqjyn62n9nyh0_0s86vmc0000gn/T/ipykernel_8017/2832893868.py:1: FutureWarning: Index.__xor__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__xor__.  Use index.symmetric_difference(other) instead.
  indA ^ indB


Int64Index([1, 2, 9, 11], dtype='int64')

<br>

# Data Indexing, Selection, and Assignment

From the numpy lecture, we already know about indexing, slicing, masking, and fancy indexing:

In [23]:
a = np.arange(16).reshape(4,4)
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]])

In [24]:
# Takes those values of the second and fourth column that are divisible by 3
a[:, [1, 3]][a[:, [1, 3]] % 3 == 0]

array([ 3,  9, 15])

Here we'll look at similar means of accessing and modifying values in Pandas Series and DataFrame objects. The corresponding patterns in Pandas are very similar to those of numpy, though there are a few quirks to be aware of.

We'll start with the simple case of the one-dimensional Series object, and then move on to the slightly more complicated two-dimensional DataFrame object.

## Data Selection in Series

As we saw in the previous section, a Series object acts in many ways like a one-dimensional NumPy array, and in many ways like a standard Python dictionary. If we keep these two overlapping analogies in mind, it will help us to understand the patterns of data indexing and selection in these arrays.

### Series as dictionary

Like a dictionary, the Series object provides a mapping from a collection of keys to a collection of values, which means most of the corresponding functions work just as well for them:

In [25]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [26]:
data.__contains__('b')

True

In [27]:
'b' in data

True

In [28]:
np.array_equal(data.keys(), data.index)

True

In [29]:
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [30]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

In [31]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

### Series as one-dimensional array

Series builds on this dictionary-like interface and provides array-style item selection via the same basic mechanisms as NumPy arrays – that is, slices, masking, and fancy indexing. Examples of these are as follows:

In [32]:
# slicing by explicit index
data['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [33]:
# slicing by implicit integer index
data[0:2] 
# Note that when slicing with an explicit index (i.e., data['a':'c']), the final index is included in the slice, 
# while when slicing with an implicit index (i.e., data[0:2]), the final index is excluded from the slice.

a    0.25
b    0.50
dtype: float64

In [34]:
(data > 0.3) & (data < 0.8)

a    False
b     True
c     True
d    False
e    False
dtype: bool

In [35]:
# masking
data[(data > 0.3) & (data < 0.8)]

b    0.50
c    0.75
dtype: float64

In [36]:
# fancy indexing
data[['a', 'e']]

a    0.25
e    1.25
dtype: float64

In [37]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=[1, 2, 3, 4])
data

1    0.25
2    0.50
3    0.75
4    1.00
dtype: float64

In [38]:
data[1:3]

2    0.50
3    0.75
dtype: float64

**If your Series has an explicit integer index, an indexing operation such as data[1] will use the explicit indices, while a slicing operation like data[1:3] will use the implicit Python-style index.**

In [39]:
data = pd.Series(['a', 'b', 'c'], index=[1, 5, 3])
data

1    a
5    b
3    c
dtype: object

In [40]:
# explicit index when indexing
data[1]

'a'

In [41]:
# implicit index when slicing
data[1:3]

5    b
3    c
dtype: object

The **loc** attribute allows indexing and slicing that *always* references the explicit index:

In [42]:
data.loc[1]

'a'

In [43]:
data.loc[1:3]

1    a
5    b
3    c
dtype: object

Note that `loc` may or may not throw Index-Errors when slicing:

In [44]:
data = pd.Series(['a', 'b', 'c'], index=[1, 5, 3])
data

1    a
5    b
3    c
dtype: object

In [45]:
try:
    
    data.loc['a':'z']
    
except KeyError:
    print(KeyError)

<class 'KeyError'>


In [46]:
try:
    
    data.loc[3:10]
    
except KeyError:
    print(KeyError)

<class 'KeyError'>


In [47]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data.loc[3:10]

3    b
5    c
dtype: object

In [48]:
data.loc['a':'z']

Series([], dtype: object)

The **iloc** attribute allows indexing and slicing that always references the implicit Python-style index:

In [49]:
data

1    a
3    b
5    c
dtype: object

In [50]:
data.iloc[1]

'b'

In [51]:
data.iloc[1:3]

3    b
5    c
dtype: object

Please, save yourself the pain and be always explicit about what you do -- always use ``.loc`` and ``.iloc``

In [52]:
%%bash
python -c "import this" | grep "Explicit"
#not saying that explicit indices are better than implicit ones, but that you should be explicit about what you're doing.

Explicit is better than implicit.


<br>

## Data Selection in DataFrame

Recall that a DataFrame acts in many ways like a two-dimensional or structured array, and in other ways like a dictionary of Series structures sharing the same index. These analogies can be helpful to keep in mind as we explore data selection within this structure.

In [53]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data = pd.DataFrame({'area':area, 'pop':pop, 'Country':'USA'})
data

,area,pop,Country
California,423967,38332521,USA
Texas,695662,26448193,USA
New York,141297,19651127,USA
Florida,170312,19552860,USA
Illinois,149995,12882135,USA


Note that if we index a DataFrame, we index the **column**!!

In [54]:
# Dictionary-style indexing results in a Series....
print(type(data["area"]))
data["area"]

<class 'pandas.core.series.Series'>


California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [55]:
# We can also dereference, though it leads to side-effects if that's actually also a method...
data.area

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [56]:
data.values

array([[423967, 38332521, 'USA'],
       [695662, 26448193, 'USA'],
       [141297, 19651127, 'USA'],
       [170312, 19552860, 'USA'],
       [149995, 12882135, 'USA']], dtype=object)

In [57]:
data.T

,California,Texas,New York,Florida,Illinois
area,423967,695662,141297,170312,149995
pop,38332521,26448193,19651127,19552860,12882135
Country,USA,USA,USA,USA,USA


For array-style indexing, Pandas again uses the loc and iloc indexers mentioned earlier. Using the iloc indexer, we can index the underlying array as if it is a simple NumPy array (using the implicit Python-style index), **but the DataFrame index and column labels are maintained in the result**:

In [58]:
#indexing the underlying numpy-array...
data.values[:3, :2]

array([[423967, 38332521],
       [695662, 26448193],
       [141297, 19651127]], dtype=object)

In [59]:
data.iloc[:3, :2]

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127


In [60]:
data

,area,pop,Country
California,423967,38332521,USA
Texas,695662,26448193,USA
New York,141297,19651127,USA
Florida,170312,19552860,USA
Illinois,149995,12882135,USA


In [61]:
data.loc[:'Illinois', :'pop']

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


In [62]:
data.loc[:,['area','pop']]

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


So, this is how we get a row!

In [63]:
data.loc["California", :]

area         423967
pop        38332521
Country         USA
Name: California, dtype: object

In [64]:
# adding a new column.. (vectorized calculations!)
data['density'] = data['pop'] / data['area']
# we can combine masking with fancy indexing
data.loc[data.density > 100, ['pop', 'density']]

,pop,density
New York,19651127,139.076746
Florida,19552860,114.806121


If you want to combine explicit and implicit indexing, you have to chain them:

In [65]:
data

,area,pop,Country,density
California,423967,38332521,USA,90.413926
Texas,695662,26448193,USA,38.018740
New York,141297,19651127,USA,139.076746
Florida,170312,19552860,USA,114.806121
Illinois,149995,12882135,USA,85.883763


In [66]:
data.iloc[1:4].loc[:, ['pop', 'density']]

,pop,density
Texas,26448193,38.018740
New York,19651127,139.076746
Florida,19552860,114.806121


**While indexing refers to columns, slicing refers to rows:**

In [67]:
data['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [68]:
data['Florida':'Illinois']

,area,pop,Country,density
Florida,170312,19552860,USA,114.806121
Illinois,149995,12882135,USA,85.883763


Again, rather be explicit about your indexing to save yourself from a lot of confusion.

In [69]:
try:
    
    data['area':'pop']

except KeyError:
    print(KeyError)

<class 'KeyError'>


In [70]:
data.loc[:, 'area':'pop']

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


Fast access to a single member using **at**

In [71]:
%%timeit
data.loc['Florida', 'pop']

4.1 µs ± 15.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [72]:
%%timeit
data.at['Florida', 'pop']

2.14 µs ± 13.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [73]:
data.loc["Texas", ["area", "pop"]]

area      695662
pop     26448193
Name: Texas, dtype: object

In [74]:
data["area"]["Texas":"Florida"]

Texas       695662
New York    141297
Florida     170312
Name: area, dtype: int64

|                        |              |                                          |                          |
|------------------------|--------------|------------------------------------------|--------------------------|
| **direct []-access**   |              |                                          |                          |
| One argument,   single | Column       | data['area']                             |                          |
| One argument,   slice  | Row          | data['Florida': 'Illinois']              | slice-top is included    |
| Both arguments         | only MultiInd| -                                        |                          |
| **.loc[]**             |              |                                          |                          |
| One argument,   single | Row          | data.loc['Florida']                      |                          |
| One argument,   slice  | Row          | data.loc['Florida': 'Illinois']          | slice-top  is included   |
| Both arguments, both   | Row, Column  | data.loc['Florida': 'Illinois', 'area']  | slice-top  is included   |
| **.iloc[]**            |              |                                          |                          |
| One argument,   single | Row          | data.iloc[0]                             |                          |
| One argument,   slice  | Row          | data.iloc[0:2]                          | slice-top  is excluded   |
| Both arguments, both   | Row, Column  | data.iloc[0: 2, 0:3]                     | slice-top  is excluded   |

### Reindexing

In [75]:
index = ['Firefox', 'Chrome', 'Safari', 'IE10', 'Konqueror']
df = pd.DataFrame({'http_status': [200, 200, 404, 404, 301],
                  'response_time': [0.04, 0.02, 0.07, 0.08, 1.0]},
                  index=index)
df

,http_status,response_time
Firefox,200,0.04
Chrome,200,0.02
Safari,404,0.07
IE10,404,0.08
Konqueror,301,1.00


In [76]:
new_index = ['Safari', 'Iceweasel', 'Comodo Dragon', 'IE10', 'Chrome']
df.reindex(new_index)

,http_status,response_time
Safari,404.0,0.07
Iceweasel,NaN,NaN
Comodo Dragon,NaN,NaN
IE10,404.0,0.08
Chrome,200.0,0.02


In [77]:
df.reindex(columns=['http_status', 'user_agent'])

,http_status,user_agent
Firefox,200,NaN
Chrome,200,NaN
Safari,404,NaN
IE10,404,NaN
Konqueror,301,NaN


**Renaming columns** 

In [78]:
df = pd.DataFrame({"a": [1, 2, 3, 4], "b": [2, 5, 7, 8]})
df = df.rename(mapper={'b': 'c'}, axis='columns')
df

,a,c
0,1,2
1,2,5
2,3,7
3,4,8


### Boolean Indexing

In [79]:
dates = pd.date_range('20130101', periods=6)
df = pd.DataFrame(np.random.randn(6,4), index=dates, columns=list('ABCD'))
df['E'] = ["one", "two", "three"] * 2
df

,A,B,C,D,E
2013-01-01,0.587937,0.027124,-0.121434,-0.164973,one
2013-01-02,-1.497560,-0.072721,-0.223712,1.518804,two
2013-01-03,0.533125,0.294877,-0.938561,-1.176123,three
2013-01-04,0.702450,1.434804,-2.449467,-0.076238,one
2013-01-05,1.181792,-0.099651,-0.030083,-0.609779,two
2013-01-06,0.358311,-1.283002,-0.252916,-0.092401,three


In [80]:
df['A'] > 0.5

2013-01-01     True
2013-01-02    False
2013-01-03     True
2013-01-04     True
2013-01-05     True
2013-01-06    False
Freq: D, Name: A, dtype: bool

In [81]:
df[df['A'] > 0.5]

,A,B,C,D,E
2013-01-01,0.587937,0.027124,-0.121434,-0.164973,one
2013-01-03,0.533125,0.294877,-0.938561,-1.176123,three
2013-01-04,0.702450,1.434804,-2.449467,-0.076238,one
2013-01-05,1.181792,-0.099651,-0.030083,-0.609779,two


In [82]:
#alternate syntax: `query` - see https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html
df.query('A > 0.5')

,A,B,C,D,E
2013-01-01,0.587937,0.027124,-0.121434,-0.164973,one
2013-01-03,0.533125,0.294877,-0.938561,-1.176123,three
2013-01-04,0.702450,1.434804,-2.449467,-0.076238,one
2013-01-05,1.181792,-0.099651,-0.030083,-0.609779,two


In [83]:
df['E'].isin(['one','two'])

2013-01-01     True
2013-01-02     True
2013-01-03    False
2013-01-04     True
2013-01-05     True
2013-01-06    False
Freq: D, Name: E, dtype: bool

In [84]:
df[df['E'].isin(['one','two'])] = np.NaN
df

,A,B,C,D,E
2013-01-01,NaN,NaN,NaN,NaN,NaN
2013-01-02,NaN,NaN,NaN,NaN,NaN
2013-01-03,0.533125,0.294877,-0.938561,-1.176123,three
2013-01-04,NaN,NaN,NaN,NaN,NaN
2013-01-05,NaN,NaN,NaN,NaN,NaN
2013-01-06,0.358311,-1.283002,-0.252916,-0.092401,three


In [85]:
pd.isna(df)

,A,B,C,D,E
2013-01-01,True,True,True,True,True
2013-01-02,True,True,True,True,True
2013-01-03,False,False,False,False,False
2013-01-04,True,True,True,True,True
2013-01-05,True,True,True,True,True
2013-01-06,False,False,False,False,False


In [86]:
pd.isna(df).any(axis=1)

2013-01-01     True
2013-01-02     True
2013-01-03    False
2013-01-04     True
2013-01-05     True
2013-01-06    False
Freq: D, dtype: bool

In [87]:
df[~df.isna().any(axis=1)]

,A,B,C,D,E
2013-01-03,0.533125,0.294877,-0.938561,-1.176123,three
2013-01-06,0.358311,-1.283002,-0.252916,-0.092401,three


In [88]:
df.dropna(how="any")

,A,B,C,D,E
2013-01-03,0.533125,0.294877,-0.938561,-1.176123,three
2013-01-06,0.358311,-1.283002,-0.252916,-0.092401,three


### Exercise

Write a Pandas snippet to get the names and scores of the people where the number of attempts in the examination is greater than 2 as a dict.

In [89]:
exam_data = {'name': ['Anastasia', 'Dima', 'Katherine', 'James', 'Emily', 'Michael', 'Matthew', 'Laura', 'Kevin', 'Jonas'],
    'score': [12.5, 9, 16.5, np.nan, 9, 20, 14.5, np.nan, 8, 19],
    'attempts' : [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],
    'qualify': ['yes', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'no', 'yes']}
labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
df = pd.DataFrame(exam_data , index=labels)
df

,name,score,attempts,qualify
a,Anastasia,12.5,1,yes
b,Dima,9.0,3,no
c,Katherine,16.5,2,yes
d,James,NaN,3,no
e,Emily,9.0,2,no
f,Michael,20.0,3,yes
g,Matthew,14.5,1,yes
h,Laura,NaN,1,no
i,Kevin,8.0,2,no
j,Jonas,19.0,1,yes


<br>

## Data assignment

In [90]:
df = pd.DataFrame({'temp_c': [17.0, 25.0]},
                  index=['Portland', 'Berkeley'])
df

,temp_c
Portland,17.0
Berkeley,25.0


### Assigning columns

In [91]:
df.loc[:,'country'] = 'USA'
df

,temp_c,country
Portland,17.0,USA
Berkeley,25.0,USA


In [92]:
df['temp_c'] <= 18

Portland     True
Berkeley    False
Name: temp_c, dtype: bool

In [93]:
df['too_cold'] = df['temp_c'] <= 18
df

,temp_c,country,too_cold
Portland,17.0,USA,True
Berkeley,25.0,USA,False


These however work in-place. To assign to a new dataframe, use `assign`:

In [94]:
df = pd.DataFrame({'temp_c': [17.0, 25.0]},
                  index=['Portland', 'Berkeley'])
df

,temp_c
Portland,17.0
Berkeley,25.0


In [95]:
df2 = df.assign(temp_f=lambda x: x.temp_c * 9 / 5 + 32)
df2

,temp_c,temp_f
Portland,17.0,62.6
Berkeley,25.0,77.0


In [96]:
#vectorized version:
df.assign(temp_f=df['temp_c'] * 9 / 5 + 32)

,temp_c,temp_f
Portland,17.0,62.6
Berkeley,25.0,77.0


In [97]:
#multiple assignments simulatenously:
df.assign(temp_f=lambda x: x['temp_c'] * 9 / 5 + 32,
          temp_k=lambda x: (x['temp_f'] +  459.67) * 5 / 9)

,temp_c,temp_f,temp_k
Portland,17.0,62.6,290.15
Berkeley,25.0,77.0,298.15


In [98]:
df

,temp_c
Portland,17.0
Berkeley,25.0


### Row assignment

In [99]:
df.loc['Berkeley', 'temp_c'] = 26.0
df

,temp_c
Portland,17.0
Berkeley,26.0


In [100]:
type(df.loc['Portland'])

pandas.core.series.Series

In [101]:
df.loc['Portland'] = pd.Series({'temp_c': 99})
df

,temp_c
Portland,99.0
Berkeley,26.0


In [102]:
df.loc['Osnabruck', 'temp_c'] = 18
df

,temp_c
Portland,99.0
Berkeley,26.0
Osnabruck,18.0


In [103]:
df = pd.concat([df, df])
df

,temp_c
Portland,99.0
Berkeley,26.0
Osnabruck,18.0
Portland,99.0
Berkeley,26.0
Osnabruck,18.0


In [104]:
df.loc['Osnabruck', 'temp_c'] = 25
df

,temp_c
Portland,99.0
Berkeley,26.0
Osnabruck,25.0
Portland,99.0
Berkeley,26.0
Osnabruck,25.0


In [105]:
df.loc['Osnabruck'] = pd.Series({'temp_c': 99})
df

ValueError: shape mismatch: value array of shape (2,) could not be broadcast to indexing result of shape (2,1)

In [ ]:
df.loc['Osnabruck']

In [106]:
np.where(df.index == 'Osnabruck')

(array([2, 5]),)

In [107]:
df.iloc[np.where(df.index == 'Osnabruck')[0][0]]

temp_c    25.0
Name: Osnabruck, dtype: float64

In [127]:
df.iloc[np.where(df.index == 'Osnabruck')[0][0]] = pd.Series({'temp_c': 100})
df

,temp_c
Portland,99.0
Berkeley,26.0
Osnabruck,100.0
Portland,99.0
Berkeley,26.0
Osnabruck,25.0


<br>

## Pandas indexing

### Multi-Indexing

While Pandas does provide objects that natively handle three-dimensional and four-dimensional data, a far more common pattern in practice is to make use of `hierarchical indexing` (also known as `multi-indexing`) to incorporate multiple index levels within a single index. In this way, higher-dimensional data can be compactly represented within the familiar one-dimensional Series and two-dimensional DataFrame objects.

In [108]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [109]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [110]:
index.names = ['state', 'year']

In [111]:
pop = pop.reindex(index)
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [112]:
pop['California', 2000], pop['California', 2010]

(33871648, 37253956)

In [113]:
pop.iloc[0]

33871648

In [114]:
pop.iloc[1]

37253956

### MultiIndex as extra dimension: stack() and unstack()

You might notice something else here: we could easily have stored the same data using a simple ``DataFrame`` with index and column labels.
In fact, Pandas is built with this equivalence in mind. The ``unstack()`` method will quickly convert a multiply indexed ``Series`` into a conventionally indexed ``DataFrame``:

In [115]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [116]:
pop.unstack()

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [117]:
index.names = [None, None]
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [118]:
pop.unstack()

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [119]:
pop.unstack().T

,California,New York,Texas
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [120]:
popdf = pop.unstack(level=0)
popdf

,California,New York,Texas
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [121]:
popdf.stack()

2000  California    33871648
      New York      18976457
      Texas         20851820
2010  California    37253956
      New York      19378102
      Texas         25145561
dtype: int64

### Index setting and resetting

Another way to rearrange hierarchical data is to turn the index labels into columns; this can be accomplished with the ``reset_index`` method.
Calling this on the population dictionary will result in a ``DataFrame`` with a *state* and *year* column holding the information that was formerly in the index.
For clarity, we can optionally specify the name of the data for the column representation:

In [122]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [123]:
pop.index.names = ['state', 'year']
print(type(pop))
pop

<class 'pandas.core.series.Series'>


state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [124]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


Often when working with data in the real world, the raw input data looks like this and it's useful to build a ``MultiIndex`` from the column values.
This can be done with the ``set_index`` method of the ``DataFrame``, which returns a multiply indexed ``DataFrame``:

In [125]:
pop_df = pop_flat.set_index(['state', 'year'])
pop_df

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [126]:
pop_df.rename_axis([None, None])

population
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [127]:
asdf = pop_df.rename_axis([None, None]).unstack()
asdf

population          
                 2000      2010
California   33871648  37253956
New York     18976457  19378102
Texas        20851820  25145561

In [128]:
asdf.columns

MultiIndex([('population', 2000),
            ('population', 2010)],
           )

In [129]:
asdf["area"] = 999
asdf

population           area
                 2000      2010     
California   33871648  37253956  999
New York     18976457  19378102  999
Texas        20851820  25145561  999

In [130]:
asdf.columns

MultiIndex([('population', 2000),
            ('population', 2010),
            (      'area',   '')],
           )

In [131]:
print(type(asdf["area"]))
asdf["area"]

<class 'pandas.core.series.Series'>


California    999
New York      999
Texas         999
Name: area, dtype: int64

In [132]:
print(type(asdf["population"]))
asdf["population"]

<class 'pandas.core.frame.DataFrame'>


,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [133]:
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [134]:
pop_df2 = pop_flat.set_index('state').rename_axis(None)
pop_df2

,year,population
California,2000,33871648
California,2010,37253956
New York,2000,18976457
New York,2010,19378102
Texas,2000,20851820
Texas,2010,25145561


In [135]:
pop_df

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [136]:
pop_df.reset_index()

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [1]:
import numpy as np
import pandas as pd

# Ufuncs and Aggregation

## Aggregation in Pandas

Aggregations are functions, where one or more dimensions of data are collapsed onto a single value, like the `max`, `sum` or `mean`- functions.

Stat-operations generally *exclude* missing data.

### For Series

In [ ]:
a = np.arange(7)
ser = pd.Series(a**2, index=a)
ser

In [ ]:
ser.sum()
#mean(), median(), min(), max(), ...

### For DataFrames

In [ ]:
df = pd.DataFrame({'A': a**2,
                   'B': a**3})
df

In [ ]:
df.mean()

In [ ]:
df.mean(axis=0)

In [ ]:
df.mean(axis='rows')

In [ ]:
df.mean(axis='columns')

The following table summarizes some other built-in Pandas aggregations:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

These are all methods of ``DataFrame`` and ``Series`` objects.

## Ufuncs


We know Ufuncs already from Numpy: It are vectorized functions that change all values of an array simultaneously. 

Pandas does the same, with a nice twist: for unary operations like negation and trigonometric functions, these ufuncs will *preserve index and column labels* in the output, and for binary operations such as addition and multiplication, Pandas will automatically *align indices* when passing the objects to the ufunc.


This means that keeping the context of data and combining data from different sources –both potentially error-prone tasks with raw NumPy arrays– become essentially foolproof ones with Pandas.

In [ ]:
rng = np.random.RandomState(0)
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

In [ ]:
np.exp(df)

### UFuncs: Index Alignment

For binary operations on two ``Series`` or ``DataFrame`` objects, Pandas will align indices in the process of performing the operation.
This is very convenient when working with incomplete data.

In [ ]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')
area

In [ ]:
population

In [ ]:
# area & population
area.index.intersection(population.index)

In [ ]:
"divide" in dir(pd.DataFrame)

In [ ]:
population/area

In [ ]:
popdens = population.divide(area, fill_value=0)
popdens

In [ ]:
popdens = popdens.replace([np.inf, -np.inf], np.nan)
popdens.dropna()

In [ ]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

In [ ]:
B = pd.DataFrame(rng.randint(0, 20, (3, 3)),
                 columns=list('ABC'))
B

In [ ]:
A+B

In [ ]:
A.add(B, fill_value=0)

### More Index-Alignment

In [ ]:
df = pd.DataFrame({'a': np.random.randint(3, size=10)}, index=np.arange(1, 20, 2))
df

Let's add a new column to this DataFrame!

In [ ]:
tmp = pd.Series([1]*len(df.index))
tmp

In [ ]:
# df.align will unify the indices
df.index.union(tmp.index)

In [ ]:
left_aligned, right_aligned = df.align(tmp, axis=0)
left_aligned

In [ ]:
right_aligned

In [ ]:
test_left_a, test_right_a = df.align(tmp, axis=0)
test_left_b, test_right_b = tmp.align(df, axis=0)
print(test_left_a.equals(test_right_b))
print(test_right_a.equals(test_left_b))

In [ ]:
tmp = pd.Series([0]*len(df.index), index=df.index)
tmp

In [ ]:
df['new'] = tmp
df

## .agg()

If you want to apply more than one operation (ufunc/aggregation), use `.agg()`:

In [ ]:
df = pd.DataFrame([[1, 2, 3],
                   [4, 5, 6],
                   [7, 8, 9],
                   [np.nan, np.nan, np.nan]],
                  columns=['A', 'B', 'C'])
df

In [ ]:
df.agg(['sum', np.min])

In [ ]:
#you can also use different aggregations for different columns: 
df.agg({'A' : ['sum', 'min'], 'B' : ['min', 'max']})

In [ ]:
#also works for ufuncs:
df.agg({'A' : 'exp', 'B' : [np.exp, 'sqrt']})

## apply()

While some ufuncs (like cumsum or exp) are pre-defined by pandas, the method `apply` can be used to run an arbitrary function on all elements of a Series or DataFrame.

In [ ]:
a = np.arange(7)
df = pd.DataFrame({'A': a**2,
                   'B': a**3})
df

In [ ]:
df.cumsum()

In [ ]:
df.apply(np.cumsum)

Using Lambda-functions, we can combine `apply` with arbitrary functions. Note that the argument of the function is always an entire column of the dataset.

In [ ]:
df.apply(lambda x: print(x, end='\n\n'))

In [ ]:
df['A'] + 1

In [ ]:
df.apply(lambda x: x+1)

In [ ]:
def my_more_complex_func(ser):
    res = [9]
    for elem in ser:
        print(elem if elem > 16 else -elem)
        res.append(elem if elem > 16 else -elem)
    return res

In [ ]:
df.apply(my_more_complex_func)

In [ ]:
df

In [ ]:
df.apply(lambda x: x.max() - x.min())

Note that `apply` works for both DataFrames and Series!

In [ ]:
df["A"].apply(lambda x: print(x,"\n"))

In [ ]:
df["A_normed"] = df["A"].apply(lambda x: x/df["A"].max())
df

It can be very useful to use dictionaries in combination with the apply-function!

In [ ]:
z_moves = {"Normal": "Breakneck Blitz", "Fighting": "All-Out Pummeling", "Flying": "Supersonic Skystrike", "Poison": "Acid Downpour", "Ground": "Tectonic Rage", "Rock": "Continental Crush", "Bug": "Savage Spin-Out", "Ghost": "Never-Ending Nightmare",
"Steel": "Corkscrew Crash", "Fire": "Inferno Overdrive", "Water": "Hydro Vortex", "Grass": "Bloom Doom", "Electric": "Gigavolt Havoc", "Psychic": "Shattered Psyche", "Ice": "Subzero Slammer", "Dragon": "Devastating Drake", "Dark": "Black Hole Eclipse", "Fairy": "Twinkle Tackle"}
df = pd.read_csv("Pokemon.csv")
df.head()

In [ ]:
df["Z-Move"] = df["Type 1"].apply(lambda x:z_moves[x])
df.head()

Using `apply`, we can also convert a list of Series into a DataFrame, by making the individual columns to Series:

In [ ]:
s = pd.Series([ ['Red', 'Green', 'White'], ['Red', 'Black'], ['Yellow']]) 
print(type(s))
s

In [ ]:
pd.Series([1, 2, 3])

In [ ]:
df = s.apply(pd.Series)
print(type(df))
df

**Note on speed:**

According to ([1]), `apply()` is twice as fast as looping through a df's `iterrows()`, and 8 times as fast as looping over python lists.

Note however, that while `apply()` is much faster at looping over the rows of your DataFrame/Series (by taking advantage of a number of internal optimizations, such as using iterators in Cython), it still inherently loops through rows. Whatever you're applying, you're still executing it once for every row. So, wherever you can make use of vectorized Ufuncs, do so - that is far more optimized and parallelized - for ([1]) exchanging the haversine distance formula with it's vectorized counterpart led to a 50-fold-decrease in time!

\(1): https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6


[1]: https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6 

### Exercise

Write a Pandas program to convert a Series of lists to one Series ("flatten" the Series).

In [ ]:
s = pd.Series([['Red', 'Green', 'White'], ['Red', 'Black'], ['Yellow']])
s

In [ ]:
# Sample Output: 
print(pd.Series(['Red', 'Green', 'White', 'Red', 'Black', 'Yellow']))
# Your code here


### Solution

In [ ]:
s.apply(pd.Series).stack().reset_index(drop=True)

# Group-By

## Split-Apply-Combine

While simple operations are already pre-defined by pandas, custom aggregations and operations can be performed via **group-by**. The group-by operation can be described as having the following steps:

* **Splitting** the data into groups based on some criteria (breaking up and grouping depending on the value of a key)
* **Applying** a function to each group independently (aggregation, transformation, filtering, ...)
* **Combining** the results into a data structure

A typical example, for where the *apply* is a summerization aggregation, is illustrated here:

![](split-apply-combine.png)

In [2]:
tmp = np.array([list("ABCABC"), np.arange(1,7)]).T
tmp

array([['A', '1'],
       ['B', '2'],
       ['C', '3'],
       ['A', '4'],
       ['B', '5'],
       ['C', '6']], dtype='<U21')

In [3]:
df = pd.DataFrame(tmp, columns=["key", "data"])
df["data"] = pd.to_numeric(df["data"])
df

,key,data
0,A,1
1,B,2
2,C,3
3,A,4
4,B,5
5,C,6


In [4]:
df.groupby("key")

Note that what is returned is not a set of `DataFrames`, but a `DataFrameGroupBy` object. This object is where the magic is: you can think of it as a special view of the `DataFrames`, which is poised to dig into the groups but does no actual computation until the aggregation is applied. This "lazy evaluation" approach means that common aggregates can be implemented very efficiently in a way that is almost transparent to the user.

To produce a result, we can apply an aggregate to this `DataFrameGroupBy` object, which will perform the appropriate apply/combine steps to produce the desired result:

In [6]:
df.groupby("key").sum().reset_index()

,key,data
0,A,5
1,B,7
2,C,9


In [7]:
df.groupby("key")["data"].sum()
# we can do column indexing just like on a normal DataFrame

key
A    5
B    7
C    9
Name: data, dtype: int64

Sometimes making a proper DataFrame out of the result requires some index name adjustments. There are usually multiple ways and it is necessary to put some thought into what you want to end up with in each situation.

In [8]:
# Sometimes making a proper DataFrame out of the result requires some index name adjustments
print(pd.DataFrame(df.groupby("key")["data"].sum().rename_axis(None)))
print(df.groupby("key")["data"].sum().reset_index())

   data
A     5
B     7
C     9
  key  data
0   A     5
1   B     7
2   C     9


### Iteration over groups

The ``GroupBy`` object supports direct iteration over the groups, returning each group as a ``Series`` or ``DataFrame``:

In [9]:
df

,key,data
0,A,1
1,B,2
2,C,3
3,A,4
4,B,5
5,C,6


In [10]:
for (key, _) in df.groupby("key"):
    print(key)
    
print()
for (_, group) in df.groupby("key"):
    print(group, "\n")

A
B
C

  key  data
0   A     1
3   A     4 

  key  data
1   B     2
4   B     5 

  key  data
2   C     3
5   C     6 



### Dispatch methods

Any method not explicitly implemented by the ``GroupBy`` object will be passed through and called on the groups, whether they are ``DataFrame`` or ``Series`` objects.
For example, you can use the ``describe()`` method of ``DataFrame``s to perform a set of aggregations that describe each group in the data:

In [11]:
df.describe()

,data
count,6.000000
mean,3.500000
std,1.870829
min,1.000000
25%,2.250000
50%,3.500000
75%,4.750000
max,6.000000


In [12]:
df.groupby("key").describe()

data                                         
    count mean      std  min   25%  50%   75%  max
key                                               
A     2.0  2.5  2.12132  1.0  1.75  2.5  3.25  4.0
B     2.0  3.5  2.12132  2.0  2.75  3.5  4.25  5.0
C     2.0  4.5  2.12132  3.0  3.75  4.5  5.25  6.0

### Exercise

The given dataset contains a column `Region` as well as a column `Pop. Density`. Write a snippet that takes as argument the dataframe containing all the countries, and returns a `Series` mapping regions to the average Population density of its countries.

In [ ]:
countries = pd.read_csv('countries.csv', index_col=0)
countries.head()

In [ ]:
countries.groupby("Region")["Pop. Density"].mean().rename_axis(None).sort_values(ascending=False)

## Aggregate, filter, transform, apply

So far, we focused on aggregation for the apply operation, but there are more options available.
In particular, ``GroupBy`` objects have ``aggregate()``, ``filter()``, ``transform()``, and ``apply()`` methods that efficiently implement a variety of useful operations before combining the grouped data.

For the purpose of the following subsections, we'll use this ``DataFrame``:

In [13]:
def create_df():
    rng = np.random.RandomState(0)
    df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                       'data1': range(6),
                       'data2': rng.randint(0, 10, 6)},
                       columns = ['key', 'data1', 'data2'])
    return df
    
df = create_df()
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


### Aggregation

While we used some *aggregate-functions* already, the function `aggregate` (or ``agg``) is the explicit version thereof.  
It can take a string, a function, or a list thereof, and compute all the aggregates at once.

In [14]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [15]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

Another useful pattern is to pass a dictionary mapping (old) column names to operations to be applied on that column:

In [17]:
df.groupby('key').agg({'data1': 'min',
                       'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


In [ ]:
df.groupby('key').aggregate({'data1': np.sum,
                             'data2': lambda x: np.std(x, ddof=1)})

### Named aggregation

To support column-specific aggregation with control over the *output* column names, pandas accepts so-called named aggregation in `GroupBy.agg()`, where:

* The keywords are the output column names
* The values are tuples whose first element is the column to select and the second element is the aggregation to apply to that column. 
    * Alternatively, you can use the pandas.NamedAgg namedtuple with the fields ['column', 'aggfunc'] to make it clearer what the arguments are. 

In [18]:
animals = pd.DataFrame({'kind': ['cat', 'dog', 'cat', 'dog'],
                        'height': [9.1, 6.0, 9.5, 34.0],
                        'weight': [7.9, 7.5, 9.9, 198.0]})
animals

,kind,height,weight
0,cat,9.1,7.9
1,dog,6.0,7.5
2,cat,9.5,9.9
3,dog,34.0,198.0


In [ ]:
assert int(pd.__version__[0]) >= 1, 'Your version of pandas is too old for this!'

In [19]:
animals.groupby("kind").agg(
       min_height = pd.NamedAgg(column='height', aggfunc='min'), # using NamedAgg
       max_height = ('height', 'max'), # less explicit alternative
       average_weight = pd.NamedAgg(column='weight', aggfunc=np.mean),
    )

,min_height,max_height,average_weight
kind,,,
cat,9.1,9.5,8.90
dog,6.0,34.0,102.75


### Filtering

A filtering operation allows you to drop data based on the group properties.
For example, we might want to keep all groups in which the standard deviation is larger than some critical value:

In [20]:
def filter_func(x):
    print(x,"\n")
    return x['data2'].std() > 4

df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [21]:
df.groupby('key').std()

,data1,data2
key,,
A,2.12132,1.414214
B,2.12132,4.949747
C,2.12132,4.242641


In [22]:
df.groupby('key').filter(filter_func)
#note that this is not an aggregate - the result has the shape of the original DataFrame, just with certain lines left out!

  key  data1  data2
0   A      0      5
3   A      3      3 

  key  data1  data2
1   B      1      0
4   B      4      7 

  key  data1  data2
2   C      2      3
5   C      5      9 



,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


### The apply() method

The ``apply()`` method lets you apply an arbitrary function to the group results.
The function should take a ``DataFrame``, and return either a Pandas object (e.g., ``DataFrame``, ``Series``) or a scalar; the combine operation will be tailored to the type of output returned.

First, remember our ``apply`` from before:

In [23]:
create_df()

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [24]:
df = create_df()
df["data1"] = df["data1"].apply(lambda x: x/df["data1"].max())
df

,key,data1,data2
0,A,0.0,5
1,B,0.2,0
2,C,0.4,3
3,A,0.6,3
4,B,0.8,7
5,C,1.0,9


Keep in mind that ``groupby`` only returns a *view of the original DataFrame*.  
Here is an ``apply()`` that normalizes the (grouped) first column with respect to each groups maximum:

In [29]:
import warnings; warnings.filterwarnings('ignore')
try:
    del newdf
except:
    pass

In [30]:
df = create_df()
maxvals = df.groupby('key')["data1"].max()
print(maxvals, '\n\n\n')
for key, group in df.groupby('key'):
    group["data1"] /= maxvals[key]
    try:
        newdf = newdf.append(group) #appending to dataframes is bad style!
    except:
        newdf = group.copy()
    print(newdf, '\n')

newdf

key
A    3
B    4
C    5
Name: data1, dtype: int64 



  key  data1  data2
0   A    0.0      5
3   A    1.0      3 

  key  data1  data2
0   A   0.00      5
3   A   1.00      3
1   B   0.25      0
4   B   1.00      7 

  key  data1  data2
0   A   0.00      5
3   A   1.00      3
1   B   0.25      0
4   B   1.00      7
2   C   0.40      3
5   C   1.00      9 



,key,data1,data2
0,A,0.00,5
3,A,1.00,3
1,B,0.25,0
4,B,1.00,7
2,C,0.40,3
5,C,1.00,9


In [31]:
def norm_by_data1(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data1'].max()
    return x

df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [32]:
df.groupby('key').apply(norm_by_data1)

,key,data1,data2
0,A,0.00,5
1,B,0.25,0
2,C,0.40,3
3,A,1.00,3
4,B,1.00,7
5,C,1.00,9


## Specifying the split key

In the simple examples presented before, we split the ``DataFrame`` on a single column name.
This is just one of many options by which the groups can be defined, and we'll go through some other options for group specification here.

### A list, array, series, or index providing the grouping keys

The keys (= new column names) can be any series or list with a length matching that of the ``DataFrame`` (the last key will relate to the last dataframe row). For example:

In [33]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [34]:
L = [0, "group 1", 0, "group 1", 2, 0]
df.groupby(L).max()

,key,data1,data2
0,C,5,9
2,B,4,7
group 1,B,3,3


### A dictionary or series mapping index to group

Another method is to provide a dictionary that maps index values to the group keys:

In [35]:
df2 = df.set_index('key')
df2

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9


In [36]:
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
df2.groupby(mapping).sum()

,data1,data2
key,,
consonant,12,19
vowel,3,8


Grouping by multiple columns forms a hierarchical index

In [37]:
df2.groupby([mapping, "key"]).sum()

data1  data2
key       key              
consonant B        5      7
          C        7     12
vowel     A        3      8

In [38]:
df = create_df()
df["key2"] = ["D", "D", "E", "D", "E", "F"]
print(df)
df.groupby(["key", "key2"]).sum()

  key  data1  data2 key2
0   A      0      5    D
1   B      1      0    D
2   C      2      3    E
3   A      3      3    D
4   B      4      7    E
5   C      5      9    F


data1  data2
key key2              
A   D         3      8
B   D         1      0
    E         4      7
C   E         2      3
    F         5      9

Video tutorial from Pycon 2015

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('5JnMutdy6Fw')

In [ ]:
import pandas as pd

# Reading Series and DataFrames

Lets have a look inside the `pokemon.csv` file and the content inside it. 
The content is quite unredable for now but this is the data we are going to use for this section of the lecture. 

In [ ]:
%%bash
head Pokemon.csv

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,,309,39,52,43,60,50,65,1,False
5,Charmeleon,Fire,,405,58,64,58,80,65,80,1,False
6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False


Pandas provides us with the function calle `read_csv` to read the csv file and take care of the data. by default it takes the first row of the csv file as column names and rest of the data belong to the columns. 

https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [ ]:
#df = pd.read_csv("Pokemon.csv",  sep= ",", header='infer', index_col=None, skiprows=None )
df = pd.read_csv("Pokemon.csv")

Imagine someboy gave you a random dataset. You don't know any of its contents. What are the first steps you do?

with `.head()` you have take a look inside the dataset. It prints the first 5 row of the dataset by default. However, you can pass number as pameter of how many data you want to view.

In [ ]:
df.head()


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [ ]:
df.shape

(800, 13)

In [ ]:
df.size

10400

similar thing can be done with `.tail()` fuction. the only difference is, it prints the data from the end of the dataset. 

In [ ]:
df.tail()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True


with `.describe()` you can print the summary of the dataset. The summary includes central tendency, 

Now what does it means? How can we interpret the results?  

as you can see in the cell avobe, the dataset has 800 entires. we can aslo verify it with `df.shape` which gives us the row and column numbers of the dataset. the row shows `count`, `mean`, `std`, `25%`, `50%`, `75%` and `max`. The coulmn has all teh columns that are possibelt to describe with in the scope of summary statistics. 

  - the **count** shows us the count of entries each column has (every column has 800 entries)
  - **mean** shows us the mean values of each column
  - **std** goes for standerddiviation. 
  - **min** values in each column
  - the **percentile** of the dataset where the data belongs to. 
  - **max** values in each column. 

In [ ]:
df.describe()

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
count,800.000000,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000
mean,362.813750,435.10250,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375
std,208.343798,119.96304,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129
min,1.000000,180.00000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000
25%,184.750000,330.00000,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000
50%,364.500000,450.00000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000
75%,539.250000,515.00000,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000
max,721.000000,780.00000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000


`.value_counts()` gives us the unique values in a column and the count of the values as well.

the clomun `Type 1` has only catagorical values.  so we get the output as categorical names and the count of the catagories.  

In [ ]:
df["Type 1"].value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         51
Electric     44
Rock         44
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

as the colum Legendary is a boolen column, it contains only true or false values. 

In [ ]:
df["Legendary"].value_counts()

False    735
True      65
Name: Legendary, dtype: int64

In [ ]:
#when working with csvs you really want to make sure if you want the first column as index-column!
df = pd.read_csv("Pokemon.csv", index_col=0)
df.tail() 

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,,
719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True
721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True


it is possible to chain multiple function calls one after another. 

In [ ]:
df.reset_index().tail()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True


In [ ]:
df.shape

(800, 12)

In [ ]:
df.reset_index().drop_duplicates(subset="#").tail()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
793,717,Yveltal,Dark,Flying,680,126,131,95,131,98,99,6,True
794,718,Zygarde50% Forme,Dragon,Ground,600,108,100,121,81,95,95,6,True
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True


In [ ]:
df = df[df['Name'] != 'Volcanion']
df.tail()

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,,
718,Zygarde50% Forme,Dragon,Ground,600,108,100,121,81,95,95,6,True
719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


when reseting index, it creates a new index column and if there are any previous index column founds, it remains there as index column, to drop the old column, which is not in any use anymore, we can use drop function and pass the column name as parameter that we want to drop and set axis as 1 to tell pandas to drop the column. 

In [ ]:
no_duplicates = df.reset_index().drop_duplicates(subset="#").reset_index().drop("index", axis=1)  
no_duplicates.tail()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
715,716,Xerneas,Fairy,NaN,680,126,131,95,131,98,99,6,True
716,717,Yveltal,Dark,Flying,680,126,131,95,131,98,99,6,True
717,718,Zygarde50% Forme,Dragon,Ground,600,108,100,121,81,95,95,6,True
718,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
719,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True


with `.to_csv()` we can save the cleaned dataframe as csv file. 


In [ ]:
no_duplicates.set_index("#").to_csv('Pokemon_no_duplicates.csv')
#no_duplicates.to_excel('Pokemon_no_duplicates.xlsx', sheet_name='Sheet1')

In [ ]:
%%bash
head Pokemon_no_duplicates.csv

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
4,Charmander,Fire,,309,39,52,43,60,50,65,1,False
5,Charmeleon,Fire,,405,58,64,58,80,65,80,1,False
6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,Squirtle,Water,,314,44,48,65,50,64,43,1,False
8,Wartortle,Water,,405,59,63,80,65,80,58,1,False
9,Blastoise,Water,,530,79,83,100,85,105,78,1,False


with `.set_index()` we can create index with our desired column

In [ ]:
gen_one = no_duplicates[no_duplicates["Generation"] == 1].set_index("#")
gen_one.tail()

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,,
147,Dratini,Dragon,NaN,300,41,64,45,50,50,50,1,False
148,Dragonair,Dragon,NaN,420,61,84,65,70,70,70,1,False
149,Dragonite,Dragon,Flying,600,91,134,95,100,100,80,1,False
150,Mewtwo,Psychic,NaN,680,106,110,90,154,90,130,1,True
151,Mew,Psychic,NaN,600,100,100,100,100,100,100,1,False


In [ ]:
first_gen_dict = gen_one["Name"].to_dict()
first_gen_dict

{1: 'Bulbasaur',
 2: 'Ivysaur',
 3: 'Venusaur',
 4: 'Charmander',
 5: 'Charmeleon',
 6: 'Charizard',
 7: 'Squirtle',
 8: 'Wartortle',
 9: 'Blastoise',
 10: 'Caterpie',
 11: 'Metapod',
 12: 'Butterfree',
 13: 'Weedle',
 14: 'Kakuna',
 15: 'Beedrill',
 16: 'Pidgey',
 17: 'Pidgeotto',
 18: 'Pidgeot',
 19: 'Rattata',
 20: 'Raticate',
 21: 'Spearow',
 22: 'Fearow',
 23: 'Ekans',
 24: 'Arbok',
 25: 'Pikachu',
 26: 'Raichu',
 27: 'Sandshrew',
 28: 'Sandslash',
 29: 'Nidoran♀',
 30: 'Nidorina',
 31: 'Nidoqueen',
 32: 'Nidoran♂',
 33: 'Nidorino',
 34: 'Nidoking',
 35: 'Clefairy',
 36: 'Clefable',
 37: 'Vulpix',
 38: 'Ninetales',
 39: 'Jigglypuff',
 40: 'Wigglytuff',
 41: 'Zubat',
 42: 'Golbat',
 43: 'Oddish',
 44: 'Gloom',
 45: 'Vileplume',
 46: 'Paras',
 47: 'Parasect',
 48: 'Venonat',
 49: 'Venomoth',
 50: 'Diglett',
 51: 'Dugtrio',
 52: 'Meowth',
 53: 'Persian',
 54: 'Psyduck',
 55: 'Golduck',
 56: 'Mankey',
 57: 'Primeape',
 58: 'Growlithe',
 59: 'Arcanine',
 60: 'Poliwag',
 61: 'Poliwhirl'

In [ ]:
[str(key)+" : "+str(val) for index, (key, val) in enumerate(first_gen_dict.items()) if index < 9]

['1 : Bulbasaur',
 '2 : Ivysaur',
 '3 : Venusaur',
 '4 : Charmander',
 '5 : Charmeleon',
 '6 : Charizard',
 '7 : Squirtle',
 '8 : Wartortle',
 '9 : Blastoise']

**Documentation!**

There are really really many arguments for this function, suiting all of your needs!

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html